# Fine-tuning Yolov3 на своих классах с помощью фреймворка darknet

In [ ]:
import cv2
import os

from PIL import Image, ImageOps
import numpy
import random

import cv2
import torch

from IPython.display import Image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

**Монтируем диск**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Клонируем репозиторий darknet-фреймворк**

In [ ]:
path = ''
cd /content/drive/MyDrive/Dataset/FineTuningYolo3

In [ ]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5937, done.
remote: Total 5937 (delta 0), reused 0 (delta 0), pack-reused 5937
Receiving objects: 100% (5937/5937), 6.35 MiB | 2.29 MiB/s, done.
Resolving deltas: 100% (3926/3926), done.
Checking out files: 100% (979/979), done.


In [ ]:
!make

make: Nothing to be done for 'all'.


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!chmod +x ./darknet

In [ ]:
!./darknet

usage: ./darknet <function>


**Preprocessing**

In [ ]:
# Готовим структуру каталогов для изображений

path_to_folder = '/content/drive/MyDrive/Dataset/FineTuningYolo3/img/' 
output_folder = '/content/drive/MyDrive/Dataset/FineTuningYolo3/img_aug/' 
img_names = os.listdir(path_to_folder)

In [ ]:
# Аугментация

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def rotate(path_to_folder, img_name, output_folder):
    im = Image.open(path_to_folder + img_name)
    im.rotate(50).save(output_folder+"rotate_"+img_name, "JPEG") 
    
def mirror(path_to_folder, img_name, output_folder):
    im = Image.open(path_to_folder + img_name)
    ImageOps.mirror(im).save(output_folder+"mirror"+img_name, "JPEG")

for img in img_names:
    rotate(path_to_folder, img, output_folder)
    mirror(path_to_folder, img, output_folder)    

In [ ]:
# Делим набор изображений на обучающую и тестовую выборку

train_list = os.listdir('./obj/')
test_samples = random.sample(train_list, k=30) 
train_samples = [i for i in train_list]
len(train_samples), len(test_samples)
f_path = './obj/'

with open('train.txt', "w") as f:
    for line in train_samples:
        f.write(f_path+line+'\n')

with open('test.txt', "w") as f:
    for line in test_samples:
        f.write(f_path+line+'\n')     

In [ ]:
# Проверяем, доступен ли GPU

is_cuda = torch.cuda.is_available()
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


**Дообучение Yolov3 на своих классах (+3 человека)**

In [ ]:
!./darknet detector train /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/data/obj.data /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/cfg/yolo-obj.cfg /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/darknet19_448.conv.23

**Детекция лица по изображению**

In [ ]:
!./darknet detector test /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/data/obj.data /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/cfg/yolo-obj.cfg /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/backup/yolo-obj_900.weights /content/drive/MyDrive/Dataset/FineTuningYolo3/JP_05853.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  32   ->   208 x 208 x  32
    2 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  64   ->   104 x 104 x  64
    4 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    5 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    6 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    7 max          2 x 2 / 2   104 x 104 x 128   ->    52 x  52 x 128
    8 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256  1.595 BFLOPs
    9 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128  0.177 BFLOPs
   10 conv    256  3 x 3 / 1    52 x  52 x 128   ->    52 x  52 x 256  1.595 BFLOPs
   11 max          2 x 2 / 2    52 x  52 x 256   ->

**Детекция лица в режиме реального времени (получаем изображение Capture)**

In [ ]:
# Подключаем веб-камеру в colab

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# По нажатию на кнопку "take_photo" сохраняется изображение в корневую папку текущего репозитория

try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
# Детекция лица в веб-камере по изображению
!./darknet detector test /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/data/obj.data /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/cfg/yolo-obj.cfg /content/drive/MyDrive/Dataset/FineTuningYolo3/darknet/backup/yolo-obj_900.weights /content/drive/MyDrive/Dataset/FineTuningYolo3/JP_05853.jpg